In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os

import numpy as np
import pandas as pd
import sklearn
from tqdm.auto import tqdm

import scipy.stats
import sklearn.neighbors

In [3]:
dataset_root_dir = 'dhs_train'

In [4]:
train_df = pd.read_csv("train.csv")


In [5]:
test_df = pd.read_csv("test_masked.csv")

In [16]:
test_df

,DHSID_EA,cname,year,lat,lon,n_asset,asset_index,cluster_id,adm1fips,adm1dhs,urban,path
0,IA-2015-7-00010009,IA,2015,9.220903,92.781530,22.0,2.721812,10009,NaN,1,R,dhs_valid/IA-2015-7-00110896.npz
1,IA-2015-7-00010011,IA,2015,7.028410,93.883430,20.0,2.287279,10011,NaN,1,R,dhs_valid/IA-2015-7-00120669.npz
2,IA-2015-7-00010017,IA,2015,12.371448,92.783665,22.0,0.677109,10017,NaN,1,R,dhs_valid/IA-2015-7-00310941.npz
3,IA-2015-7-00010044,IA,2015,11.727304,92.719257,21.0,1.793683,10044,NaN,1,R,dhs_valid/IA-2015-7-00332819.npz
4,IA-2015-7-00010060,IA,2015,9.185310,92.777645,22.0,2.758168,10060,NaN,1,R,dhs_valid/IA-2015-7-00030310.npz
...,...,...,...,...,...,...,...,...,...,...,...,...
2670,IA-2015-7-00360403,IA,2015,17.410860,78.558810,19.0,2.823787,360403,NaN,36,U,dhs_valid/IA-2015-7-00100002.npz
2671,IA-2015-7-00360454,IA,2015,17.450098,78.360138,22.0,1.990244,360454,NaN,36,R,dhs_valid/IA-2015-7-00040683.npz
2672,IA-2015-7-00360474,IA,2015,17.973616,79.597408,20.0,2.633020,360474,NaN,36,U,dhs_valid/IA-2015-7-00200056.npz
2673,IA-2015-7-00360476,IA,2015,18.956406,77.934361,21.0,0.429279,360476,NaN,36,R,dhs_valid/IA-2015-7-00030367.npz


In [6]:
display(train_df.head())

,DHSID_EA,cname,year,lat,lon,n_asset,asset_index,n_water,water_index,cluster_id,adm1fips,adm1dhs,urban,path
0,IA-2015-7-00010004,IA,2015,9.165413,92.742696,22.0,2.650768,22.0,5.000000,10004,NaN,1,R,dhs_train/IA-2015-7-00192065.npz
1,IA-2015-7-00010005,IA,2015,8.307356,93.093792,22.0,2.157784,22.0,5.000000,10005,NaN,1,R,dhs_train/IA-2015-7-00330893.npz
2,IA-2015-7-00010007,IA,2015,7.016968,93.893226,21.0,1.832751,21.0,4.619048,10007,NaN,1,R,dhs_train/IA-2015-7-00140183.npz
3,IA-2015-7-00010016,IA,2015,9.194938,92.800432,22.0,2.746096,22.0,5.000000,10016,NaN,1,R,dhs_train/IA-2015-7-00261454.npz
4,IA-2015-7-00010018,IA,2015,8.055606,93.543892,22.0,2.581869,22.0,5.000000,10018,NaN,1,R,dhs_train/IA-2015-7-00191303.npz


In [7]:
feature_cols = ["lat", "lon"]
label_cols = ['water_index']

In [8]:
k = 10
knn = sklearn.neighbors.KNeighborsRegressor(n_neighbors=k, weights='uniform', algorithm='auto')

In [9]:
train_X = train_df[feature_cols]
train_Y = train_df[label_cols]
test_X = test_df[feature_cols]

In [10]:
train_X

,lat,lon
0,9.165413,92.742696
1,8.307356,93.093792
2,7.016968,93.893226
3,9.194938,92.800432
4,8.055606,93.543892
...,...,...
18717,17.184983,79.989186
18718,17.465781,78.418820
18719,17.939231,79.512509
18720,17.420175,78.491013


In [11]:
knn.fit(train_X, train_Y)
preds = knn.predict(test_X)

In [12]:
solution_df = test_df.copy()
solution_df["water_index"] = preds

In [13]:
filtered_solution_df = solution_df[["DHSID_EA", "water_index"]]
filtered_solution_df.to_csv("solution.csv")

In [14]:
filtered_solution_df

,DHSID_EA,water_index
0,IA-2015-7-00010009,5.000000
1,IA-2015-7-00010011,4.961905
2,IA-2015-7-00010017,4.435498
3,IA-2015-7-00010044,4.976190
4,IA-2015-7-00010060,5.000000
...,...,...
2670,IA-2015-7-00360403,4.921531
2671,IA-2015-7-00360454,5.000000
2672,IA-2015-7-00360474,4.739351
2673,IA-2015-7-00360476,4.807677
